# Libraries

In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
import re

# Web Scrapping

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.mohfw.gov.in/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[-1]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(36)

,Name of State / UT,Total Confirmed cases*,Cured/Discharged/Migrated,Deaths**
0,Andaman and Nicobar Islands,33,33,0
1,Andhra Pradesh,3110,1896,56
2,Arunachal Pradesh,2,1,0
3,Assam,526,62,4
4,Bihar,2730,749,13
5,Chandigarh,238,186,3
6,Chhattisgarh,291,72,0
7,Dadar Nagar Haveli,2,0,0
8,Delhi,14053,6771,276
9,Goa,67,19,0


# Data Cleaning

In [6]:
# date-time information
# =====================

# today's date
now  = datetime.now()
# format date to month-day-year
df_bs['Date'] = now.strftime("%m/%d/%Y") 

# add 'Date' column to dataframe
df_bs['Date'] = pd.to_datetime(df_bs['Date'], format='%m/%d/%Y')

# df_bs.head(36)

In [7]:
# remove extra characters from 'Name of State/UT' column
df_bs['Name of State / UT'] = df_bs['Name of State / UT'].str.replace('#', '')

In [8]:
# latitude and longitude information
# ==================================

# latitude of the states
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 'Maharashtra':19.7515,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670, 'Dadar Nagar Haveli' : 20.1809, 'Sikkim':27.5330}

# longitude of the states
long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662, 'Dadar Nagar Haveli' : 73.0169, 'Sikkim':88.5122}

# add latitude column based on 'Name of State / UT' column
df_bs['Latitude'] = df_bs['Name of State / UT'].map(lat)

# add longitude column based on 'Name of State / UT' column
df_bs['Longitude'] = df_bs['Name of State / UT'].map(long)

# df_bs.head(36)

In [9]:
# unique state names
df_bs['Name of State / UT'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadar Nagar Haveli', 'Delhi', 'Goa', 'Gujarat',
       'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand',
       'Karnataka', 'Kerala', 'Ladakh', 'Madhya Pradesh', 'Maharashtra',
       'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha',
       'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu',
       'Telengana', 'Tripura', 'Uttarakhand', 'Uttar Pradesh',
       'West Bengal'], dtype=object)

In [10]:
# number of missing values 
df_bs.isna().sum()

Name of State / UT           0
Total Confirmed cases*       0
Cured/Discharged/Migrated    0
Deaths**                     0
Date                         0
Latitude                     0
Longitude                    0
dtype: int64

In [11]:
# number of unique values 
df_bs.nunique()

Name of State / UT           35
Total Confirmed cases*       33
Cured/Discharged/Migrated    31
Deaths**                     20
Date                          1
Latitude                     35
Longitude                    31
dtype: int64

# Saving data

In [12]:
# saving data
# ===========

# file names as year-month-day.csv format
file_name = now.strftime("%Y_%m_%d")+'.csv'

# location for saving the file
file_loc = 'C:\\Users\\imdevskp\\Documents\\github\\covid_india\\.day_by_day_data\\'

# save file as a scv file
df_bs.to_csv(file_loc + file_name, index=False)

df_bs.head(36)

,Name of State / UT,Total Confirmed cases*,Cured/Discharged/Migrated,Deaths**,Date,Latitude,Longitude
0,Andaman and Nicobar Islands,33,33,0,2020-05-26,11.7401,92.6586
1,Andhra Pradesh,3110,1896,56,2020-05-26,15.9129,79.7400
2,Arunachal Pradesh,2,1,0,2020-05-26,28.2180,94.7278
3,Assam,526,62,4,2020-05-26,26.2006,92.9376
4,Bihar,2730,749,13,2020-05-26,25.0961,85.3131
5,Chandigarh,238,186,3,2020-05-26,30.7333,76.7794
6,Chhattisgarh,291,72,0,2020-05-26,21.2787,81.8661
7,Dadar Nagar Haveli,2,0,0,2020-05-26,20.1809,73.0169
8,Delhi,14053,6771,276,2020-05-26,28.7041,77.1025
9,Goa,67,19,0,2020-05-26,15.2993,74.1240


In [13]:
# column names 
# df_bs.columns

# Combining data

In [14]:
# list of all files available
# ! ls C:\Users\imdevskp\Documents\github\covid_india\.day_by_day_data

In [15]:
# location of the file
loc = "C:\\Users\\imdevskp\\Documents\\github\\covid_india\\.day_by_day_data\\"

# list of all files
files = glob.glob(loc+'2020*.csv')
   
# container for each day's data's dataframe
dfs = []

# loop through the files and append to the dfs list
for i in files:
    # read data
    df_temp = pd.read_csv(i)
    
    # rename columns
    df_temp = df_temp.rename(columns={'Cured':'Cured/Discharged'})
    df_temp = df_temp.rename(columns={'Cured/Discharged':'Cured/Discharged/Migrated', 
                                      'Total Confirmed cases *': 'Total Confirmed cases', 
                                      'Total Confirmed cases ': 'Total Confirmed cases', 
                                      'Total Confirmed cases* ': 'Total Confirmed cases'})
    df_temp = df_temp.rename(columns=lambda x: re.sub('Total Confirmed cases \(Including .* foreign Nationals\) ',
                                                      'Total Confirmed cases',x))
    df_temp = df_temp.rename(columns=lambda x: re.sub("Death.*", "Death", x))

    
    # append to the df_s
    dfs.append(df_temp)
    
# print(dfs)

# concat dataframes
complete_data = pd.concat(dfs, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)


complete_data['Death'] = complete_data['Death'].astype('str').str.extract('(\d+)').astype('int')

complete_data.sample(10)

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
454,2020-04-16,Ladakh,NaN,NaN,10,34.2996,78.2932,0,17
1614,3/20/2020,Uttar Pradesh,18.0,1.0,9,26.8467,80.9462,0,19
307,2020-04-09,Telengana,NaN,NaN,35,18.1124,79.0193,7,427
771,2020-04-25,Uttarakhand,NaN,NaN,25,30.0668,79.0193,0,48
268,2020-04-08,West Bengal,NaN,NaN,13,22.9868,87.8550,5,99
1644,3/21/2020,Haryana,3.0,14.0,0,29.0588,76.0856,0,17
1639,3/21/2020,Jammu and Kashmir,4.0,0.0,0,33.7782,76.5762,0,4
1299,2020-05-16,Mizoram,NaN,NaN,1,23.1645,92.9376,0,1
324,2020-04-10,Delhi,NaN,NaN,25,28.7041,77.1025,12,720
2137,5/23/2020,Tamil Nadu,NaN,NaN,7128,11.1271,78.6569,98,14753


In [16]:
# complete_data.columns

## Preprocessing

In [17]:
# fix datatype
complete_data['Date'] = pd.to_datetime(complete_data['Date'])

# sort rows
complete_data = complete_data.sort_values(['Date', 'Name of State / UT']).reset_index(drop=True)

# fill missing values with 0
cols = ['Total Confirmed cases (Indian National)', 'Total Confirmed cases ( Foreign National )', 
              'Cured/Discharged/Migrated', 'Death']
complete_data[cols] = complete_data[cols].fillna(0).astype('int')

In [18]:
# rename state/UT names
complete_data['Name of State / UT'].replace('Chattisgarh', 'Chhattisgarh', inplace=True)
complete_data['Name of State / UT'].replace('Pondicherry', 'Puducherry', inplace=True) 

In [19]:
# select only rows with more than 1 case
complete_data = complete_data[complete_data['Total Confirmed cases']>0]

## Final dataframe

In [20]:
# random rows
complete_data.sample(3)

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
1502,2020-05-01,West Bengal,0,0,139,22.9868,87.8550,33,795
183,2020-03-17,Odisha,1,0,0,20.9517,85.0985,0,1
2282,2020-05-25,Odisha,0,0,550,20.9517,85.0985,7,1336


In [21]:
# complete data info
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2320 entries, 0 to 2327
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Date                                        2320 non-null   datetime64[ns]
 1   Name of State / UT                          2320 non-null   object        
 2   Total Confirmed cases (Indian National)     2320 non-null   int32         
 3   Total Confirmed cases ( Foreign National )  2320 non-null   int32         
 4   Cured/Discharged/Migrated                   2320 non-null   int32         
 5   Latitude                                    2320 non-null   float64       
 6   Longitude                                   2320 non-null   float64       
 7   Death                                       2320 non-null   int32         
 8   Total Confirmed cases                       2320 non-null   int64         
dtypes: dateti

## Save as .csv file

In [22]:
# save data in a csv file
complete_data.to_csv('complete.csv', index=False)

In [23]:
# complete_data.groupby('Date').count()

In [24]:
# complete_data.sort_values('Death', ascending=False)

In [25]:
complete_data[complete_data['Date']=='2020-05-25']

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
2259,2020-05-25,Andaman and Nicobar Islands,0,0,33,11.7401,92.6586,0,33
2260,2020-05-25,Andhra Pradesh,0,0,1856,15.9129,79.7400,56,2823
2261,2020-05-25,Arunachal Pradesh,0,0,1,28.2180,94.7278,0,1
2262,2020-05-25,Assam,0,0,55,26.2006,92.9376,4,378
2263,2020-05-25,Bihar,0,0,702,25.0961,85.3131,13,2587
2264,2020-05-25,Chandigarh,0,0,186,30.7333,76.7794,3,238
2265,2020-05-25,Chhattisgarh,0,0,67,21.2787,81.8661,0,252
2266,2020-05-25,Dadar Nagar Haveli,0,0,0,20.1809,73.0169,0,2
2267,2020-05-25,Delhi,0,0,6540,28.7041,77.1025,261,13418
2268,2020-05-25,Goa,0,0,19,15.2993,74.1240,0,66
